<a href="https://colab.research.google.com/github/SabigBenmumin/SF323_hw/blob/main/SF323_CN408_HW4_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW4 - RAG (5 points)

---

ในการบ้านนี้เราจะมาฝึกฝนการทำ RAG โดยการบ้านจะแบ่งเป็นขั้นตอนต่อไปนี้

1. Generate synthetic questions: สร้างคำถามจากข้อมูลที่มีเพื่อไว้ทดสอบระบบ

2. Building RAG

3. Evaluate RAG

    3.1  Evaluate the retriever

    3.2  Evaluate the answer

การบ้านนี้ยาวแนะนำให้เริ่มต้นแต่เนิ่นๆ

## 0. Setup

In [3]:
!pip3 install pinecone


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: C:\Users\Sabig\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [20]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
from google.colab import userdata
from openai import OpenAI
import random
import re
from pinecone import Pinecone, ServerlessSpec

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('openrouter'),
)

def generate(prompt: str):
    response = client.chat.completions.create(
        model="google/gemini-2.5-flash-lite",
        messages = [{'role':'user',
                     'content':prompt}]
    )
    return response.choices[0].message.content

generate("Hello")

'Hello there! How can I help you today?'

## 1. Generate synthetic questions (0.5 points)

ใน part แรก เราจะมาสร้าง Question Answer pairs เพื่อใช้สำหรับตรวจสอบ RAG. โดยทรัพยากรที่จำกัด เราจะสร้างแค่ 30 คู่เท่านั้น โดยในการสร้าง คำถามเราจะใช้ LLM (gemini-2.5-flash-lite) สร้าง และ เราจะใช้ LLM มาตรวจสอบคำถามว่าเหมาะสมไหม

1.1 โหลดข้อมูลที่จะใช้ทำ RAG. ข้อมูลเป็นหนังสือ Biochemistry จาก [MedQA](https://github.com/jind11/MedQA)

In [3]:
!wget https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/Biochemistry_Lippincott.txt

--2025-08-31 14:07:36--  https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/Biochemistry_Lippincott.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1353650 (1.3M) [text/plain]
Saving to: ‘Biochemistry_Lippincott.txt’

Biochemistry_Lippin 100%[===================>]   1.29M  --.-KB/s    in 0.04s   

2025-08-31 14:07:36 (29.9 MB/s) - ‘Biochemistry_Lippincott.txt’ saved [1353650/1353650]



1.2 โหลด data แล้วตัดให้เป็น chunk ด้วย recursivesplitting

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('Biochemistry_Lippincott.txt', 'r') as file:
    biochem_textbook = file.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Big chunk for generating questions
    chunk_overlap=100,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

doc_chunks = text_splitter.split_text(biochem_textbook)

1.3 สร้าง prompt สำหรับ QA

In [5]:
def gen_QA_generation_prompt(context):

    QA_generation_prompt = f"""You are a biology professor creating exam questions from a biochemistry textbook.

Your task is to write a factual exam question and answer based on the provided context.

Context:
<context>
{context}
</context>

Provide your response in JSON format:
{{
  "question": "your_question",
  "answer": "your_answer",
  "reference": "the part of the context where the answer come from"
}}

**Important Requirements**:
- The question must be answerable with specific, concise factual information from the context
- The question must be self-contained (no references to "the passage" or "the context" such as "According to the context" or "According to the provided information")
- The answer must come directly from the provided context.
- The reference should include only the key part without anything extra like headers or numbers.
- The reference MUST match exactly the part of the context without any extra strings or ...
- Keep the answer concise


If the context is unsuitable for creating exam questions without knowing the context, return: "BAD CONTEXT"
"""
    return QA_generation_prompt

In [22]:
# Testing
# context = random.sample(doc_chunks, 1)[0] # ลองทดสอบ ด้วย context ที่ แรนด้อม เช่น sdfsdfad ดูว่าโมเดลตอบ "BAD CONTEXT" ไหม
context = random.sample("dfjslkaf;dlafd", 1)[0] # ลองทดสอบ ด้วย context ที่ แรนด้อม เช่น sdfsdfad ดูว่าโมเดลตอบ "BAD CONTEXT" ไหม
qa_prompt = gen_QA_generation_prompt(context)
response = generate(qa_prompt)

print(f'Context: {context}\n')
print(response)

Context: l

BAD CONTEXT


1.4 สร้าง prompt สำหรับเช็ค คำถาม

In [7]:
def gen_eval_question_prompt(context, question):
    eval_question_prompt = f"""Given the following context and exam question, evaluate whether the question meets these criteria:

1. **Groundedness**: Can the question be answered using only the provided context?
2. **Relevance**: Is the question a suitable exam question that tests important concepts and in the question format?
3. **Stand-alone**: Can someone with domain knowledge understand and answer this question **without seeing this specific context**? The question must not include something like "According to the context..." or "According to the provided information ...".

Context:
<context>
{context}
</context>

Question: {question}

Think carefully about each criteria. Then, respond with Yes or No for each criterion in JSON format:
{{
  "groundedness": "Yes/No",
  "relevance": "Yes/No",
  "stand_alone": "Yes/No"
}}
"""
    return eval_question_prompt

In [8]:
# Helper function to extract json from the output
def extract_json_from_output(output_string):
    try:
        return json.loads(output_string)
    except:
        # Find content between ```json and ```
        pattern = r'```json\s*(.*?)\s*```'
        match = re.search(pattern, output_string, re.DOTALL)

        if match:
            json_str = match.group(1)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {e}")
                return None
    return None

In [9]:
# Testing
clean_response = json.loads(response.replace("```json","").replace("```",""))
eval_prompt = gen_eval_question_prompt(context, clean_response['question'])
result = generate(eval_prompt)

print(eval_prompt)
print('-----')
print(result)
print('-----')
print(extract_json_from_output(result))

Given the following context and exam question, evaluate whether the question meets these criteria:

1. **Groundedness**: Can the question be answered using only the provided context?
2. **Relevance**: Is the question a suitable exam question that tests important concepts and in the question format?
3. **Stand-alone**: Can someone with domain knowledge understand and answer this question **without seeing this specific context**? The question must not include something like "According to the context..." or "According to the provided information ...".

Context:
<context>
4.

Argininosuccinate cleavage: Argininosuccinate is cleaved by argininosuccinate lyase to yield arginine and fumarate. The arginine serves as the immediate precursor of urea. The fumarate is hydrated to malate, providing a link with several metabolic pathways. Malate can be oxidized by malate dehydrogenase to oxaloacetate, which can be transaminated to aspartate (see Fig. 19.8) and enter the urea cycle (see Fig. 19.14). 

1.5 สร้างคำถามจนกว่า จะได้คำถามที่ผ่านเกณฑ์ทั้งหมด 30 คำถาม

### **Code ด้านล่างสร้าง eval dataset ซึ่งไม่ต้องรัน เพราะ Data สร้างไว้ให้แล้ว**

สามารถกดดูได้ถ้าสนใจ

In [ ]:
NUM_QUESTIONS = 30

random.seed(77)
questions = []
count = 1
while len(questions) != NUM_QUESTIONS:
    count += 1
    # randomly draw a chunk
    context = random.sample(doc_chunks, 1)[0]

    # Gen a question-answer pair
    qa_prompt = gen_QA_generation_prompt(context)
    answer = generate(qa_prompt)

    if answer == "BAD CONTEXT":
        continue
    clean_answer = json.loads(answer.replace("```json","").replace("```",""))

    # Check the question-answer pair
    eval_prompt = gen_eval_question_prompt(context, clean_answer['question'])
    result = generate(eval_prompt)
    json_result = extract_json_from_output(result)
    if json_result is None:
        continue

    if (json_result['groundedness'] == 'Yes' and
          json_result['relevance'] == 'Yes' and
          json_result['stand_alone'] == 'Yes'):

        clean_answer['context'] = context
        questions.append(clean_answer)

print(f"Number of attempts: {count}")
df_questions = pd.DataFrame(questions)
df_questions.to_csv('RAG_eval_df.csv')
df_questions

**Note**: อันนี้คือการสร้าง basic questions ซึ่งมันจะขึ้นอยู่กับแค่ chunk เดียว ในความจริง คำถามอาจจะต้องใช้หลาย chunks มาช่วยตอบ   

### คำถามสำหรับ Part 1

**Q1.1** (0.15 points): ข้อจำกัดของ synthetic data ตามแบบด้านบนนี้มีอะไรบ้าง คำถามแบบไหนที่จะไม่มีอยู่ในชุดข้อมูลด้านบน

**Answer**: คำถามที่สร้างขึ้นจะอิงจาก chunk เดียวเท่านั้น ไม่สามารถสร้างคำถามที่ต้องใช้ข้อมูลจากหลาย chunks มาประกอบกันตอบ

**Q1.2**: (0.15 points) Prompt สำหรับสร้าง QA มี Output ออกมาประกอบไปด้วยอะไรบ้าง และแต่ส่วนมีการ prompt อย่างไร จงอธิบาย

**Answer**: ประกอบด้วย question, answer, context
prompt แบบการใช้ json, กำหนด requirements ชัดเจน

**Q1.3**: (0.2 points) เกณฑ์ในการเช็ค คำถามจาก context ที่ให้มีอะไรบ้างจงอธิบาย และ ให้คิดเกณฑ์เพิ่มอีกอย่างหนึ่งที่ควรจะเช็คเพิ่ม

**Answer**:
มี 3 เกณฑ์
- Groundedness
- Relevance
- Stand alone

เพิ่มเกณท์
เพิ่มการตรวจสอบว่าคำถามมีความชัดเจนและเฉพาะเจาะจงพอที่จะมีคำตอบที่แน่นอน ไม่คลุมเครือหรือตีความได้หลายแบบ

## 2. Building our RAG

สำหรับส่วนนี้เราจะมาสร้าง RAG pipeline กัน

In [23]:
# Load the data again in case you start looking from here
!wget https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/Biochemistry_Lippincott.txt

--2025-08-31 15:33:18--  https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/Biochemistry_Lippincott.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1353650 (1.3M) [text/plain]
Saving to: ‘Biochemistry_Lippincott.txt.2’

Biochemistry_Lippin 100%[===================>]   1.29M  --.-KB/s    in 0.05s   

2025-08-31 15:33:19 (26.0 MB/s) - ‘Biochemistry_Lippincott.txt.2’ saved [1353650/1353650]



In [24]:
with open('Biochemistry_Lippincott.txt', 'r') as file:
    biochem_textbook = file.read()

### 2.1 Preparing the data

- ในขั้นตอนถัดไปเราจะมาเตรียมข้อมูลให้พร้อม ก่อนที่จะเอาไป upend ไปที่ database
- ขั้นตอนหลักคือเราจะต้องแบ่งบทความเป็นส่วนย่อๆแทนที่จะใช้ทั้งบทความไป embed
- เราจะทดสอบสองแบบ และจะใช้ `Langchain` library มาช่วย
 - แบบแรกคือ ตัดเป็นความยาวเท่าๆกัน ใช้ `CharacterTextSplitter` ([Documentation](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.CharacterTextSplitter.html))
 - แบบสองคือ ตัดแบบrecursiveตาม structure ใช้ `RecursiveCharacterTextSplitter` ([Documentation](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html))
- parameter ที่สำคัญคือ `chunk` หรือความยาวของประโยคที่จะตัด


- ลองทดสอบกับบทความแรก

In [12]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
chunk_size = 500 # Test different numbers
chunk_overlab = 50

char_splitter = CharacterTextSplitter(chunk_size = chunk_size,
                                      chunk_overlap=chunk_overlab,
                                      separator='', #character that you would like to split on
                                      strip_whitespace=True)

char_chunks = char_splitter.split_text(biochem_textbook)

print(len(char_chunks))
char_chunks[:10]

3000


['For additional ancillary materials related to this chapter, please visit thePoint.\n\nI. OVERVIEW\n\nProteins are the most abundant and functionally diverse molecules in living systems. Virtually every life process depends on this class of macromolecules. For example, enzymes and polypeptide hormones direct and regulate metabolism in the body, whereas contractile proteins in muscle permit movement. In bone, the protein collagen forms a framework for the deposition of calcium phosphate crystals, acti',
 'the deposition of calcium phosphate crystals, acting like the steel cables in reinforced concrete. In the bloodstream, proteins, such as hemoglobin and albumin, transport molecules essential to life, whereas immunoglobulins fight infectious bacteria and viruses. In short, proteins display an incredible diversity of functions, yet all share the common structural feature of being linear polymers of amino acids. This chapter describes the properties of amino acids. Chapter 2 explores how

- Default separators สำหรับ `RecursiveCharacterTextSplitter` นั้นมี แค่ `["\n\n", "\n", " ", ""]` เราเลยเพิ่ม `.` ลงไปด้วย

In [13]:
chunk_size = 500 # Test different numbers
chunk_overlap = 50

recur_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,
                                                chunk_overlap=chunk_overlap,
                                                strip_whitespace=True,
                                                separators=["\n\n", "\n", ".", " ", ""]
                                                )
recur_chunks = recur_splitter.split_text(biochem_textbook)

print(len(recur_chunks))
recur_chunks[:10]

4139


['For additional ancillary materials related to this chapter, please visit thePoint.\n\nI. OVERVIEW',
 'Proteins are the most abundant and functionally diverse molecules in living systems. Virtually every life process depends on this class of macromolecules. For example, enzymes and polypeptide hormones direct and regulate metabolism in the body, whereas contractile proteins in muscle permit movement. In bone, the protein collagen forms a framework for the deposition of calcium phosphate crystals, acting like the steel cables in reinforced concrete',
 '. In the bloodstream, proteins, such as hemoglobin and albumin, transport molecules essential to life, whereas immunoglobulins fight infectious bacteria and viruses. In short, proteins display an incredible diversity of functions, yet all share the common structural feature of being linear polymers of amino acids. This chapter describes the properties of amino acids',
 '. Chapter 2 explores how these simple building blocks are joined to 

- สามารถทดสอบดูได้ที่ web นี้ https://chunkviz.up.railway.app/

### Creating splitted document (0.25 points)
- ถัดไปเราจะสร้างจริง เราจะตัดบทความทั้งหมดสองแบบเพื่อไว้ทดสอบว่าแบบไหนดีกว่า   
  - แบบแรกคือ `CharacterTextSpliter`
  - แบบสองคือ `RecursiveTextSpliter`
- `chunk_size` เราจะตั้งไว้ที่ 500 และ `chunk_overlab` เป็น 50


In [14]:
chunk_size = 500
chunk_overlab = 50

char_splitter = CharacterTextSplitter(chunk_size = chunk_size,
                                      chunk_overlap=chunk_overlab,
                                      separator='',
                                      strip_whitespace=True)

char_chunks = char_splitter.split_text(biochem_textbook)

recur_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,
                                                chunk_overlap=chunk_overlap,
                                                strip_whitespace=True,
                                                separators=["\n\n", "\n", ".", " ", ""]
                                                )

recur_chunks = recur_splitter.split_text(biochem_textbook)

split_docs = {'char': char_chunks,
              'recur': recur_chunks}

In [15]:
len(split_docs['char'])

3000

In [16]:
len(split_docs['recur'])

4139

### 2.2 Embedding Data

- **Note**: ต้องใช้ Hugging Face Token ([here](https://huggingface.co/docs/hub/en/security-tokens)) สามารถตั้งเป็น secret key ใน colab ได้

In [17]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

เราจะทดสอบ Embeddding สองตัว ดังต่อไปนี้
- 'all-MiniLM-L6-v2'
- 'BAAI/bge-m3' เป็นตัวที่ดีของ SentenceTransformer แต่ว่าขนาดใหญ่กว่า (See [Documentation](https://huggingface.co/BAAI/bge-m3))

เนื่องด้วย documents ที่เราใช้นั้นใหญ่ เราจะใช้ cuda ในการรัน
- ไปที่ Runtime > Change runtime type > T4 GPU


In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('No cuda!! - Embedding time will be very long!!!')

mini_embedding =  SentenceTransformer('all-MiniLM-L6-v2',  device=device)
bge_embedding =  SentenceTransformer('BAAI/bge-m3',  device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

ทดสอบ Embedding

In [25]:
query = 'Today is a nice day.'
mini_ec = mini_embedding.encode(query)
bge_ec = bge_embedding.encode(query)

print(mini_ec[:5])
print(bge_ec[:5])

[-0.0337642   0.07306819  0.09585132 -0.01667641 -0.0293686 ]
[-0.01927846  0.02027403 -0.05240517  0.00278672 -0.0184157 ]


### 2.3 Creating Embedded Documents (0.25 points)

- ขั้นตอนถัดไป เราจะมาสร้าง Embedded Documents สำหรับแต่ละ splited docs (`char` and `recur`) และ embedding (`mini`, `bge`)
- เพราะฉะนั้นจะมีด้วยกันทั้งหมด สี่อัน
- **Note**: Code น่าจะรันนาน โดยเฉพาะถ้าคุณไม่ได้ใช้ GPU (มากกว่า 20 นาที)

In [27]:
embedded_docs = { 'mini-char' : mini_embedding.encode(char_chunks),
                  'mini-recur': mini_embedding.encode(recur_chunks),
                  'bge-char'  : bge_embedding.encode(char_chunks),
                  'bge-recur' : bge_embedding.encode(recur_chunks)}

## Pinecone Database (1 points)
- ในสวนนี้เราจะสร้าง pinecone database
- ขั้นแรกคุณจะต้องไปสมัครและเอา api มาใส่ให้เรียบร้อย
- pinecone webiste: https://www.pinecone.io/

- สร้าง 4 index สำหรับ 4 embedded_docs ที่เราสร้างไว้ และเก็บไว้ใน dict `indexes`

**Note**: Free tier ใช้ได้แค่ 5 index

In [32]:
pinecone = Pinecone(api_key=userdata.get('pinecone_key')) # from colab secret key

# Store the index in the dict
indexes = {}

embeds = {'mini': mini_embedding,
          'bge': bge_embedding}

for doc in embedded_docs:
    index_name = 'hw4-rag' + doc
    # Cleaning up the index
    if index_name in [index.name for index in pinecone.list_indexes()]:
          pinecone.delete_index(index_name)

    # Creating a serverless index
    pinecone.create_index(
        name = index_name,  ## fill in here
        dimension = embeds[doc.split('-')[0]].get_sentence_embedding_dimension(),
        metric = 'cosine', ## fill in here
        spec = ServerlessSpec(cloud='aws', region='us-east-1'))

    indexes[doc] = pinecone.Index(index_name) ## fill in here

- หลังจากสร้างเสร็จ เอาข้อมูลไปใส่บน database ตามที่สร้างไว้
- ให้ unsertที่ละ 200 chunks ต่อครั้ง pinecone มีข้อจำกัด ในการ unsert

In [33]:
batch_size = 200

for doc in tqdm(embedded_docs):
    for i in range(0, len(embedded_docs[doc]), batch_size):
        # find end of batch
        i_end = min(i+batch_size, len(embedded_docs[doc]))

        # create IDs batch
        ids = [str(x) for x in range(i, i_end)]
        # create metadata batch
        metadatas = [{'text': text} for text in split_docs[doc.split('-')[1]][i:i_end]]
        # create embeddings
        em_chunk = embedded_docs[doc][i:i_end]

        # create records list for upsert
        records = []
        for x in range(len(ids)):
            ## **Fill in your code below**
            records.append({
                'id': ids[x],
                'values': em_chunk[x].tolist(),
                'metadata': metadatas[x]
            })




        # upsert to Pinecone
        indexes[doc].upsert(vectors=records)

  0%|          | 0/4 [00:00<?, ?it/s]

- ตรวจสอบ pinecone database ว่าเรียบร้อยก่อนจะไปต่อ
- code ข้างบนควรรันแค่ครั้งเดียว หลังจากนั้นไม่จำเป็นต้อง upsert อีก เรียกใช้ได้เลย

### RAG functions (1 points)

- Load the data for eval

In [34]:
!wget https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/RAG_eval_df.csv

--2025-08-31 16:09:58--  https://raw.githubusercontent.com/yongsa-nut/SF323_CN408_AIEngineer/refs/heads/main/RAG_eval_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33160 (32K) [text/plain]
Saving to: ‘RAG_eval_df.csv’

RAG_eval_df.csv     100%[===================>]  32.38K  --.-KB/s    in 0.008s  

2025-08-31 16:09:58 (4.15 MB/s) - ‘RAG_eval_df.csv’ saved [33160/33160]



In [35]:
eval_df = pd.read_csv('RAG_eval_df.csv')

- Code box ด้านล่าง โหลด all indexes ถ้ามีข้อมูลอยู่แล้วไม่จำเป็นต้องรัน code ด้านบนเพื่อ upsert ใหม่

In [38]:
# Load all indexes
embedded_list = ['mini-char', 'mini-recur', 'bge-char','bge-recur' ]
embeds = {'mini': mini_embedding,
          'bge': bge_embedding}

pinecone = Pinecone(api_key=userdata.get('pinecone_key'))

INDEX_NAME = 'hw4-rag'
indexs = {}
for doc in embedded_list:
    indexs[doc] = pinecone.Index(INDEX_NAME + doc)

- เติม code ในช่องด้านล่าง เพื่อ retrieve documents จาก vector database
  1. เอา query ไป embed ด้วย `embed_model`
  2. เอา query ไปดึง chunks ที่เก็บไว้ใน vector database (index)
  3. ดึง metadata (text) ออกมา เก็บไว้ใน List
  4. return list นั้นออกไป

In [40]:
def retrieve_docs(query, embed_model, index, top_k):
    ## Embedding the query
    embed_query = embed_model.encode(query).tolist()

    ## Retrieve documents
    retrieved_docs = index.query(vector=embed_query,
                                 top_k=top_k,
                                 include_metadata=True)

    # Get the actual text
    texts = [r['metadata']['text'] for r in retrieved_docs['matches']]
    return texts

- เติม code ในช่องด้านล่างให้สร้าง function ที่รับ คำถาม, embedding model, database index, top_k
  1. ไปหาว่า documents ที่ใกล้คำถามที่สุดคืออะไร
  2. เอา documents ที่ได้มาสร้าง prompt เพื่อตอบคำถาม
  3. เอา prompt ไป gen response แล้วก็ return response ออกมา

In [42]:
def RAG_response(query, embed_model, index, top_k=3):
    # return the response from the model with augmented prompt
    retrieved_docs = retrieve_docs(query, embed_model, index, top_k)
    context = "\n\n".join(retrieved_docs)
    prompt = f"""You are a biochemistry expert. Answer the following question based on the provided context.

Context:
{context}

Question: {query}

Provide a clear, concise answer based only on the information in the context."""
    response = generate(prompt)
    return response


In [43]:
# Test
query = eval_df['question'][0]
response = RAG_response(query, embeds['bge'], indexs['bge-char'])

print('Response: ', response)
print('\nGround truth: ',eval_df['answer'][0])

Response:  Phosphorylation.

Ground truth:  Phosphorylation of eIF-2


## 3. Evaluate our RAG (1 point)

เพื่อให้ eval เหมือนกัน เราจะใช้ data ที่มีมาให้แล้ว

### 3.1 Evaluate the Retrieval

- สิ่งที่เราต้องการทดสอบมีด้วยกันทั้งหมด 3 อย่าง แต่ละอย่างมี สองค่า
  - embedding model: `mini` หรือ `bge`
  - spliting method: `char` หรือ `recur`
  - top-k: `1` หรือ `5`
- สิ่งที่เราจะคำนวณ คือ recall = relevant retrieve / total relevant. คำถามนั้นเราสร้างเอง และเรารู้ว่า มีแค่ chunk เดียวจากทั้งหมด ที่ relevant ดังน้้น ในแต่ละคำถาม ผลที่ได้จะเป็น 0 หรือ 1 เท่านั้น พูดอีกอย่างคือที่ดึงมามี context (เช็คจาก reference) หรือไม่

- ดังนั้นจะมีทั้งหมด 8 ค่า เราจะสร้างรูปมาวนตรวจสอบและเก็บค่าของทั้งหมดไว้ใน `DataFrame` ดังนั้น `DataFrame` นี้จะมี 4 columns: `embedding, spliting, top-k, score` และมี 8 rows

- **Note**: reference มาจากโมเดล ซึ่งโมเดลอาจจะไม่ตัดมาเป๊ะจาก context. ในการเช็ค reference ใช้ exact match up to a threshold เพื่อความง่ายและเร็ว ถ้า reference มีความซับซ้อน ควรจะใช้ตำแหน่งของข้อความ (line) หรือ ใช้โมเดลมาตรวจสอบ

In [44]:
combinations = [(em, split, k) for em in ['mini','bge'] for split in ['char','recur'] for k in [1, 5]]
retrieval_results = []
threshold = 20

for (em, split, k) in tqdm(combinations):
    total = 0
    for index, row in eval_df.iterrows():
        question = row['question']
        retrieved_docs = retrieve_docs(question,
                                       embeds[em],
                                       indexs[em+'-'+split],
                                       k)

        context = "\n".join(retrieved_docs)
        # Check if the reference is in the retreive docs
        if row['reference'][:threshold] in context:
            total += 1

    retrieval_results.append({'embedding':em,
                    'splitting':split,
                    'top-k':k,
                    'avg_score':total/len(eval_df)})

retrieval_results_df = pd.DataFrame(retrieval_results)

retrieval_results_df

  0%|          | 0/8 [00:00<?, ?it/s]

,embedding,splitting,top-k,avg_score
0,mini,char,1,0.466667
1,mini,char,5,0.933333
2,mini,recur,1,0.633333
3,mini,recur,5,0.933333
4,bge,char,1,0.466667
5,bge,char,5,0.966667
6,bge,recur,1,0.766667
7,bge,recur,5,0.966667


### 3.2 Evaluate answers (1 points)

- สิ่งที่เราต้องการทดสอบมีด้วยกันทั้งหมด 3 อย่าง แต่ละอย่างมี สองค่า
  - embedding model: `mini` หรือ `bge`
  - spliting method: `char` หรือ `recur`
  - top-k: `1` หรือ `5`
- ดังนั้นจะมีทั้งหมด 8 ค่า เราจะสร้างรูปมาวนตรวจสอบและเก็บค่าของทั้งหมดไว้ใน `DataFrame` ดังนั้น `DataFrame` นี้จะมี 4 columns: `embedding, spliting, top-k, score` และมี 8 rows
- เราใช้ training มาตรวจสอบ
- ถัดไปในการ eval นี้ เราจะใช้ LLM มาตรวจว่าคำตอบถูกต้องสมบูรณ์ไหน
- **Task**: สิ่งสำคัญคือ eval prompt สำหรับ LLM as a judge โดยที่ prompt ที่จะให้สร้างมีข้อกำหนดดังนี้
  - เราจะตรวจสองแค่สิ่งเดียวคือ ความถูกต้องของคำตอบ
  - คะแนนที่ได้จาก prompt จะต้องเป็นตัวเลข 0 - 4. 0 คือน้อยสุด (ไม่ถูกต้องเลย) 4 คือมากสุด (ถูกต้องครบถ้วน)
  - ตัวเลข จะต้องอยู่ใน <answer> tags
- หลังจากตรวจครบแล้วให้หาคะแนนเฉลี่ย และเก็บค่านั้นไว้ ใน column `score`.
- สุดท้าย print `DataFrame` ออกมา แบบไหนทำได้ดีที่สุด?
- **Note**:
  - ใช้เวลาในการรันประมาณ 12 นาที
  - ข้อมูลที่เอามาทดสอบ เป็น public data ในเรื่องที่ค่อนข้างจะมีข้อมูลเยอะ (bio) โมเดลน่าจะมีความรู้เพียงพอที่จะได้เกือบหมดด้วยตัวเอง

In [46]:
combinations = [(em, split, k) for em in ['mini','bge'] for split in ['char','recur'] for k in [1, 5]]

RAG_results = []

for (em, split, k) in tqdm(combinations):
      avg = 0
      for i in range(len(eval_df)):
          ## Get the response from RAG_response
          response = RAG_response(eval_df['question'][i],
                                  embeds[em],
                                  indexs[em+'-'+split],
                                  k)

          eval_prompt = f'''Your prompt here
          '''

          answer = generate(eval_prompt)

          # Extract the number in <answer> tags
          match = re.search(r'<answer>(\d+)</answer>', answer)
          if match:
              avg += int(match.group(1))

      RAG_results.append({'embedding':em,
                      'splitting':split,
                      'top-k':k,
                      'avg_score':avg/len(eval_df)})

RAG_results = pd.DataFrame(RAG_results)
RAG_results

  0%|          | 0/8 [00:00<?, ?it/s]

,embedding,splitting,top-k,avg_score
0,mini,char,1,0.0
1,mini,char,5,0.0
2,mini,recur,1,0.0
3,mini,recur,5,0.0
4,bge,char,1,0.0
5,bge,char,5,0.0
6,bge,recur,1,0.0
7,bge,recur,5,0.0
